In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize, differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

import itertools
from collections import Counter

In [4]:
potential = 'AHO'
cutoff = 4
shots = 1024

In [5]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [6]:
num_qubits

3

In [7]:
min_eigenvalue

np.complex128(-0.16478526068502247+0j)

In [124]:
operator_pool = []
for i in range(num_qubits):
    operator_pool.append(qml.Y(wires=[i]))
    operator_pool.append(qml.Z(wires=[i]))
    #operator_pool.append(qml.X(wires=[i]))

operator_pool = [qml.Hamiltonian([1.0], [x]) for x in operator_pool]

In [162]:
def tensor_product(ops):
    """Compute the tensor product of a list of operators using the @ operator."""
    result = ops[0]
    for op in ops[1:]:
        result = result @ op
    return result

def pad_operator(single_qubit_op, target_wire, num_qubits):
    # Build a list of operators for all qubits.
    ops = []
    for j in range(num_qubits):
        if j == target_wire:
            ops.append(single_qubit_op(wires=j))
        else:
            ops.append(qml.Identity(wires=j))
    # Return the tensor product of these operators.
    return tensor_product(ops)

operator_pool = []
for i in range(num_qubits):
    # Pad the Y operator for qubit i.
    padded_Y = pad_operator(qml.Y, i, num_qubits)
    operator_pool.append(qml.Hamiltonian([1.0], [padded_Y]))
    
    # Pad the Z operator for qubit i.
    padded_Z = pad_operator(qml.Z, i, num_qubits)
    operator_pool.append(qml.Hamiltonian([1.0], [padded_Z]))


In [163]:
# Define the Hamiltonian for the CRY gate:
# H = 1/2*(I⊗Y) - 1/2*(Z⊗Y)
c_pool = []
coeffs = [0.5, -0.5]
for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                obs = [qml.Identity(control) @ qml.PauliY(target), qml.PauliZ(control) @ qml.PauliY(target)]
                cry = qml.Hamiltonian(coeffs, obs)
                c_pool.append(cry)

In [164]:
operator_pool = operator_pool #+ c_pool
operator_pool

[1.0 * (Y(0) @ I(1) @ I(2)),
 1.0 * (Z(0) @ I(1) @ I(2)),
 1.0 * (I(0) @ Y(1) @ I(2)),
 1.0 * (I(0) @ Z(1) @ I(2)),
 1.0 * (I(0) @ I(1) @ Y(2)),
 1.0 * (I(0) @ I(1) @ Z(2))]

In [171]:
if potential == 'DW':
    basis_state = [0]*num_qubits
else:
    basis_state = [1] + [0]*(num_qubits-1)

#basis_state = [1]*(num_qubits)
basis_state = [0]*num_qubits

In [172]:
pauli_hamiltonian = qml.pauli_decompose(H)
ham_paulis = pauli_hamiltonian.ops
ham_coeffs = pauli_hamiltonian.coeffs

In [173]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def op_check_circuit(params, op_list, observable):

    qml.BasisState(basis_state, wires=range(num_qubits))
    
    idx=0
    for op in op_list:
        qml.ApproxTimeEvolution(op, time=params[idx], n=1)
        idx+=1

    return qml.expval(observable)

In [174]:
def commutator_expectation(params, op_list, O):
    total = 0.0
    for pauli, coeff in zip(ham_paulis, ham_coeffs):

        OHP = (coeff*pauli) @ O
        HPO = O @ (coeff*pauli) 
       
        exp1 = op_check_circuit(params, op_list, HPO)
        exp2 = op_check_circuit(params, op_list, OHP)
        total += (exp1 - exp2)

    return total


In [175]:
dev2 = qml.device("default.qubit", wires=num_qubits,shots=shots)
@qml.qnode(dev2)
def cost_function(params, op_list):

    qml.BasisState(basis_state, wires=range(num_qubits))
    
    idx=0
    for op in op_list:
        qml.ApproxTimeEvolution(op, time=params[idx], n=1)
        idx+=1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [176]:
pool = operator_pool.copy()
op_list = []
op_params = []

for _ in range(1):
    com_list = []
    for O in pool:
        # Compute the expectation value of the commutator.
        comm_exp = commutator_expectation(op_params, op_list, O)
        com_list.append((O, np.abs(comm_exp)))
        print((O, np.abs(comm_exp)))

    max_op, max_grad = max(com_list, key=lambda x: x[1])
    print(f"Max op is {max_op}, with grad: {max_grad}")
    print('#########################################')

(1.0 * (Y(0) @ I(1) @ I(2)), tensor(0., requires_grad=True))
(1.0 * (Z(0) @ I(1) @ I(2)), tensor(0., requires_grad=True))
(1.0 * (I(0) @ Y(1) @ I(2)), tensor(0., requires_grad=True))
(1.0 * (I(0) @ Z(1) @ I(2)), tensor(0., requires_grad=True))
(1.0 * (I(0) @ I(1) @ Y(2)), tensor(0., requires_grad=True))
(1.0 * (I(0) @ I(1) @ Z(2)), tensor(0., requires_grad=True))
Max op is 1.0 * (Y(0) @ I(1) @ I(2)), with grad: 0.0
#########################################


In [53]:
num_steps = 2
op_list = []
op_params = []
energies = []

#variables
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

pool = operator_pool.copy()
success = False

for i in range(num_steps):

    print("########################################")
    print(f"step: {i}")


    if i != 0:
        print(f"Removing {max_op} from pool")
        pool.remove(max_op)
    
    com_list = []
    for O in pool:
        # Compute the expectation value of the commutator.
        comm_exp = commutator_expectation(op_params, op_list, O)
        com_list.append((O, abs(comm_exp)))

    max_op, max_grad = max(com_list, key=lambda x: x[1])
    print(f"Max op is {max_op}, with grad: {max_grad}")
    op_list.append(max_op)

    # Generate Halton sequence
    num_dimensions = len(op_list) + 1
    num_samples = popsize
    halton_sampler = Halton(d=num_dimensions)
    halton_samples = halton_sampler.random(n=num_samples)
    scaled_samples = 2 * np.pi * halton_samples

    bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
    x0 = np.concatenate((op_params, np.array([np.random.random()*2*np.pi])))
    
    res = differential_evolution(cost_function,
                                    bounds,
                                    x0=x0,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples,
                                    )
    
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    energies.append(min_e)

    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            pre_op_params.tolist().pop()
            success = True
            break
        if abs(min_eigenvalue-min_e) < 1e-6:
                success = True
                break


    

########################################
step: 0
Max op is Z(2), with grad: 0.9613786418865984


c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\pennylane\ops\op_math\composite.py:209: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)
c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\autoray\autoray.py:1127: ComplexWarning: Casting complex values to real discards the imaginary part
  return x.astype(dtype, **kwargs)


Min E: 0.4238165462812194
False
########################################
step: 1
Removing Z(2) from pool
Max op is Y(2), with grad: 4.0245584642661925e-16


KeyboardInterrupt: 

In [50]:
op_list

[Z(2), Y(2)]

In [114]:
dev = qml.device("default.qubit", wires=4)
@qml.qnode(dev)
def circuit():
    qml.ApproxTimeEvolution(max_op, time=0.0, n=1)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [263]:
dev = qml.device("default.qubit", wires=num_qubits, shots=2)
@qml.qnode(dev)
def final_circuit(params):

    basis_state = [0,0,0,0]
    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        o = type(op)
        o(params[params_index], wires=op.wires)
        params_index += 1

    return qml.state()

In [264]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
print(qml.draw(final_circuit)(x0))

0: ─╭|Ψ⟩─────────────────────────────────────────────────────────────┤  State
1: ─├|Ψ⟩──RY(0.71)─╭RX(4.59)──RY(3.68)───────────────────────────────┤  State
2: ─├|Ψ⟩──RY(0.03)─│───────────────────╭RX(1.83)──RY(0.33)───────────┤  State
3: ─╰|Ψ⟩──RY(4.11)─╰●─────────RY(5.20)─╰●─────────RY(5.78)──RZ(5.39)─┤  State


In [ ]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o) for o in op_list],
        }

In [ ]:
data

In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\TimeEv\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
#variables
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
atol = 1e-3
popsize = 20

num_steps = 5
op_list = []
op_params = []

for i in range(num_steps):

    print(f"step: {i}")

    grad_list = []

    for op in operator_pool:
        grad = compute_grad(op, op_list, op_params)
        grad_list.append(abs(grad))

    maxidx = np.argmax(grad_list)
    op_list.append(operator_pool[maxidx])

    bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]
    res = differential_evolution(cost_function,
                                    bounds,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=atol,
                                    strategy=strategy,
                                    popsize=popsize
                                    )
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    print("Testing CZ pool")
    cz_e = []
    for term in cz_pool:
        energy = circuit(op_params, op_list, try_cz=True, cz_wires=term.wires)
        cz_e.append(energy)

    min_cz_e = cz_e[np.argmin(cz_e)]
    min_cz_term = cz_pool[np.argmin(cz_e)]
    if min_cz_e < min_e:
        print(f"Adding {min_cz_term} reduces energy further")
        op_list.append(min_cz_term)
        min_e = min_cz_e
        print(f"Min E: {min_e}")
    
    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            op_params = pre_op_params
            break


    